In [ ]:
import time
import threading
import paho.mqtt.client as mqtt
import pygame
import csv
import json

## Script para toma de datos automatizada

In [ ]:
AUDIO_FILE = "audio_for_test/audio_test_10min.wav"
MODEL_NAME = "model_NNA_float32"

# Variables globales
recording = False
results = []            # Aquí se almacenan las inferencias del baby_monitor
audio_start_time = None
last_voltage = None     # Último valor de voltaje recibido
last_current = None     # Último valor de corriente recibido
last_power = None       # Último valor de potencia recibido

# Configuración del cliente MQTT
def on_connect(client, userdata, flags, rc):
    print("Conectado al broker MQTT, código de retorno:", rc)
    client.subscribe("Bedroom/sensor/baby_monitor")

def on_message(client, userdata, msg):
    global recording, audio_start_time, last_voltage, last_current, last_power

    payload_str = msg.payload.decode('utf-8').strip()
    current_time_ms = int(time.time() * 1000)
    relative_timestamp = (current_time_ms - audio_start_time) if audio_start_time else 0

    if msg.topic == "Bedroom/sensor/baby_monitor" and recording:
        # Formato esperado: "detected_state, inference_time_ms, output_class_0, output_class_1, used_ram"
        fields = [f.strip() for f in payload_str.split(',')]
        if len(fields) == 5:
            entry = {
                "timestamp":         relative_timestamp,
                "detected_state":    fields[0],
                "inference_time_ms": int(fields[1]),
                "output_class_0":    float(fields[2]),
                "output_class_1":    float(fields[3]),
                "used_ram":          int(fields[4]),
            }
            results.append(entry)
            print("Inferencia registrada:", entry)
        else:
            print("Formato inesperado baby_monitor:", payload_str)

# Inicialización del cliente MQTT
client = mqtt.Client()
client.username_pw_set("Broker username", "Broker password") # Insertar  aqui las credenciales del broker MQTT
client.on_connect = on_connect
client.on_message = on_message
client.connect("Broker IP", 1883, 60)  # Insertar aqui la IP del broker MQTT

def run_mqtt():
    client.loop_forever()

mqtt_thread = threading.Thread(target=run_mqtt)
mqtt_thread.start()

time.sleep(2)  # Esperar conexión MQTT

# Reproducir el audio y comenzar la toma de datos
print("Cargando y reproduciendo audio:", AUDIO_FILE)
pygame.mixer.init()
pygame.mixer.music.load(AUDIO_FILE)

audio_start_time = int(time.time() * 1000)
pygame.mixer.music.play()
recording = True

while pygame.mixer.music.get_busy():
    time.sleep(1)

recording = False
print("Reproducción finalizada, deteniendo la recogida de datos...")

client.disconnect()
mqtt_thread.join()

# Guardar los resultados en CSV
csv_filename = f"../Results/model_results_new/{MODEL_NAME}_results.csv"
fieldnames = [
    "timestamp",
    "detected_state",
    "inference_time_ms",
    "output_class_0",
    "output_class_1",
    "used_ram",
]

with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for entry in results:
        writer.writerow(entry)

print("Datos recogidos guardados en:", csv_filename)
